In [1]:
import numpy as np
import pandas as pd
import glob

In [3]:
# read subj id list
subids = pd.read_csv('~/GIT/ENIGMA/data/SUB_ids.csv', header=None)
#print(subids)

# read in aparc volumes, thickness, area
rh_vol_aparc = pd.read_table('~/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/aseg_aparc/aparc_rh_stats_volume.txt', header=0)
lh_vol_aparc = pd.read_table('~/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/aseg_aparc/aparc_lh_stats_volume.txt', header=0)
rh_thick = pd.read_table('~/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/aseg_aparc/aparc_rh_stats_thickness.txt', header=0)
lh_thick = pd.read_table('~/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/aseg_aparc/aparc_lh_stats_thickness.txt', header=0)
rh_sa = pd.read_table('~/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/aseg_aparc/aparc_rh_stats_area.txt', header=0)
lh_sa =  pd.read_table('~/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER_ENIGMA/aseg_aparc/aparc_lh_stats_area.txt', header=0)


ids = lh_thick['lh.aparc.thickness']
lst = [e[5:] for e in ids]
lh_thick['lh.aparc.thickness']=lst

lh_thick=lh_thick.rename(columns={"lh.aparc.thickness": "sub_names"})

ids = rh_vol_aparc['rh.aparc.volume']
lst = [e[5:] for e in ids]
rh_vol_aparc['rh.aparc.volume']=lst
rh_vol_aparc=rh_vol_aparc.rename(columns={"rh.aparc.volume": "sub_names"})


ids = lh_vol_aparc['lh.aparc.volume']
lst = [e[5:] for e in ids]
lh_vol_aparc['lh.aparc.volume']=lst
lh_vol_aparc=lh_vol_aparc.rename(columns={"lh.aparc.volume": "sub_names"})


In [11]:
names=[]
counter=0
sub_names=[]
for file in glob.glob("/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER/aseg_aparc/aseg_stats/*"):
    print('test')
    print(file)
    sub_name = str(file[69:81])
    if counter == 0:
        base = list(pd.read_table(file, header=0).columns)
    values = np.array(pd.read_table(file))
    base = np.vstack((base, values))
    sub_names.append(sub_name)
    counter = counter +1

#basecols = base[0,:] #colnames
base =pd.DataFrame(base)
base = base.drop(0, axis=0)  # drop row that is just the column names
base.columns = basecols
base['sub_names']=sub_names
first_column = base.pop('sub_names')
base.pop('Measure:volume')
# insert column using insert(position,column_name,
# first_column) function
base.insert(0, 'sub_names', first_column)



KeyError: '[0] not found in axis'

In [343]:
tmp=base.merge(lh_vol_aparc, left_on='sub_names', right_on='sub_names')
enigma_aparc_aseg_volume=tmp.merge(rh_vol_aparc, left_on='sub_names', right_on='sub_names')
enigma_aparc_aseg_volume = enigma_aparc_aseg_volume.sort_values(by='sub_names')
enigma_aparc_aseg_volume.to_csv('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/FREESURFER/aparc_aseg_lr_volume.csv')


In [335]:
hcpstats = pd.read_csv('~/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/HCPAGING/hcpaging_aparc+aseg_volume_stats.tsv', header=0,  sep = '\t')


zscore = pd.DataFrame(columns = hcpstats.columns)

for col in hcpstats.columns:
    if col =='Subject':
        continue
    zscore.loc[0,col]=np.mean(hcpstats.loc[:,col])
    zscore.loc[1,col]=np.std(hcpstats.loc[:,col])


In [336]:
zscores_all =  pd.DataFrame(columns = hcpstats.columns)

for col in hcpstats.columns:
    if col =='Subject':
        continue
    try:
        tmp = (enigma_aparc_aseg_volume.loc[:,col].astype(float)-zscore.loc[0,col])/zscore.loc[1,col]
        print(col)
        print(np.mean(tmp))
        zscores_all.loc[:,col]=(enigma_aparc_aseg_volume.loc[:,col].astype(float)-zscore.loc[0,col])/zscore.loc[1,col]
    except:         
        print(col + ' does not exist')



Left-Lateral-Ventricle
0.4918692645342541
Left-Inf-Lat-Vent
0.8690595503703498
Left-Cerebellum-White-Matter
0.139233066913834
Left-Cerebellum-Cortex
-1.3031409166409442
Left-Thalamus-Proper
-0.1511237108045977
Left-Caudate
0.4319939724520308
Left-Putamen
0.4393761088878222
Left-Pallidum
-2.292231368381251
3rd-Ventricle
0.8374060139710502
4th-Ventricle
0.4555946910770238
Brain-Stem
-0.8014578592412848
Left-Hippocampus
-0.382894219071195
Left-Amygdala
-0.07170405773151767
CSF
0.7393864675802789
Left-Accumbens-area
-0.5736258319574561
Left-VentralDC
-1.1628124745927104
Left-vessel
1.1007202434281824
Left-choroid-plexus
4.101855895432257
Right-Lateral-Ventricle
0.5983579727996124
Right-Inf-Lat-Vent
0.5426786272823363
Right-Cerebellum-White-Matter
0.4493777803673643
Right-Cerebellum-Cortex
-1.1756235989348804
Right-Thalamus-Proper
-0.8424421425717941
Right-Caudate
0.1751223924839759
Right-Putamen
0.07479584261288268
Right-Pallidum
-1.9713748137896345
Right-Hippocampus
-0.42724768433987825
R

In [337]:
zscores_all['Subject']=enigma_aparc_aseg_volume['sub_names']

In [339]:
zscores_all.to_csv('/Users/emilyolafson/GIT/ENIGMA/data/FREESURFER/zscores_aseg_aparc_vol_ENIGMA_HCPaging.csv')